In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/11/22 19:17:11 INFO SparkContext: Running Spark version 2.4.3
21/11/22 19:17:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/22 19:17:11 INFO SparkContext: Submitted application: 0245594e-3924-4e36-841e-fb1debe967b5
21/11/22 19:17:11 INFO SecurityManager: Changing view acls to: oscar
21/11/22 19:17:11 INFO SecurityManager: Changing modify acls to: oscar
21/11/22 19:17:11 INFO SecurityManager: Changing view acls groups to: 
21/11/22 19:17:11 INFO SecurityManager: Changing modify acls groups to: 
21/11/22 19:17:11 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
21/11/22 19:17:11 INFO Utils: Successfully started service 'sparkDrive

21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:32927/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1637605032261
21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:32927/jars/mainargs_2.12-0.1.4.jar with timestamp 1637605032261
21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:32927/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1637605032261
21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:32927/jars/os-lib_2.12-0.7.1.j

21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:32927/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1637605032267
21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:32927/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1637605032267
21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:32927/jars/argonaut_2.12-6.2.3.jar with timestamp 1637605032267
21/11/22 19:17:12 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@72108b3a
import spark.implicits._


defined function run

In [64]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .where(col("year") === 1995)

load at cmd63.sc:4

1 / 1

races: Dataset[Row] = [raceId: string, year: string ... 6 more fields]

In [65]:
val lapTimeToMs = (time: String) => {
    val regex = """([0-9]|[0-9][0-9]):([0-9][0-9])\.([0-9][0-9][0-9])""".r
    time match {
        case regex(min,sec,ms) => min.toInt * 60 * 1000 + sec.toInt * 1000 + ms.toInt
        case "\\N" => 180000
    }
    
}: Long

lapTimeToMs: String => Long = ammonite.$sess.cmd64$Helper$$Lambda$5871/1415776713@47588dc7

In [66]:
val lapTimeToMsUDF = udf(lapTimeToMs)
spark.udf.register("lapTimeToMs", lapTimeToMsUDF)

lapTimeToMsUDF: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd64$Helper$$Lambda$5871/1415776713@47588dc7,
  LongType,
  Some(List(StringType))
)
res65_1: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd64$Helper$$Lambda$5871/1415776713@47588dc7,
  LongType,
  Some(List(StringType))
)

In [67]:
val msToLapTime = (time: Long) => {
    val mins = time / 60000
    val secs = (time - mins*60000)/1000
    val ms = time - mins*60000 - secs*1000
    
    val formattedSecs = if((secs / 10).toInt == 0) "0" + secs else secs
    // if ms = 00x -> "0"+"0"+x . if ms = 0xx -> "0"+ms
    val formattedMs = if((ms / 100).toInt == 0) "0" + (if((ms / 10).toInt == 0) "0" + ms else ms) else ms
    mins + ":" + formattedSecs + "." + formattedMs    
}: String

msToLapTime: Long => String = ammonite.$sess.cmd66$Helper$$Lambda$5879/731447006@6ef342bb

In [68]:
val msToLapTimeUDF = udf(msToLapTime)
spark.udf.register("msToLapTime", msToLapTimeUDF)

msToLapTimeUDF: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd66$Helper$$Lambda$5879/731447006@6ef342bb,
  StringType,
  Some(List(LongType))
)
res67_1: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd66$Helper$$Lambda$5879/731447006@6ef342bb,
  StringType,
  Some(List(LongType))
)

In [69]:
val avg_season_lap_times = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")
    .withColumnRenamed("time", "lapTime")
    .join(races, Seq("raceId"), "right")
    .withColumn("milliseconds", col("milliseconds").cast(IntegerType))
    .groupBy("driverId", "raceId").avg("milliseconds")
    .withColumnRenamed("avg(milliseconds)", "avgLapInMs")
    .withColumn("avgLapTime", msToLapTimeUDF(col("avgLapInMs").cast(LongType)))

load at cmd68.sc:4

1 / 1

avg_season_lap_times: DataFrame = [driverId: string, raceId: string ... 2 more fields]

In [70]:
val drivers = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/drivers.csv")

load at cmd69.sc:4

1 / 1

drivers: DataFrame = [driverId: string, driverRef: string ... 7 more fields]

In [71]:
val mostConsistentDriver = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")
    .join(races, Seq("raceId"), "right")
    .select("raceId", "driverId", "position", "fastestLapTime")
    .withColumn("fastestLapMs", lapTimeToMsUDF(col("fastestLapTime")))
    .join(avg_season_lap_times, Seq("raceId", "driverId"), "left")
    .withColumn("diffLapTimes", abs('avgLapInMs - 'fastestLapMs).cast(IntegerType))
    .groupBy("driverId").avg("diffLapTimes")
    .orderBy("avg(diffLapTimes)")
    .withColumn("avg_diff_lap_times", msToLapTimeUDF(col("avg(diffLapTimes)").cast(IntegerType)))
    .join(drivers, "driverId")
    // drop jerome dAmbrosio as he only participated in 1 race
    .filter(col("driverId") =!= 816)
    .withColumn("driver", concat(col("forename"), lit(" "), col("surname")))
    .select("driver", "avg_diff_lap_times")

load at cmd70.sc:4

1 / 1

mostConsistentDriver: DataFrame = [driver: string, avg_diff_lap_times: string]

In [72]:
mostConsistentDriver.show(300)

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd71.sc:1

...

show at cmd71.sc:1

1 / 1

show at cmd71.sc:1

1 / 1

+--------------------+------------------+
|              driver|avg_diff_lap_times|
+--------------------+------------------+
|         Pedro Diniz|              null|
|      Jos Verstappen|              null|
|  Michael Schumacher|              null|
|          Pedro Lamy|              null|
|     Karl Wendlinger|              null|
|       Jan Magnussen|              null|
|Heinz-Harald Fren...|              null|
|      Martin Brundle|              null|
|      Roberto Moreno|              null|
|   Gianni Morbidelli|              null|
|       Mika Häkkinen|              null|
|          Jean Alesi|              null|
|       Olivier Panis|              null|
|     Bertrand Gachot|              null|
|Domenico Schiatta...|              null|
|       Ukyo Katayama|              null|
|        Eddie Irvine|              null|
|          Taki Inoue|              null|
|   Pierluigi Martini|              null|
|  Massimiliano Papis|              null|
|      Gerhard Berger|            

In [21]:
mostConsistentDriver.write.mode("overwrite").parquet("../out/consistency2012.parquet")

run at ThreadPoolExecutor.java:1128

1 / 1

run at ThreadPoolExecutor.java:1128

1 / 1

parquet at cmd20.sc:1

...

64 / 64

parquet at cmd20.sc:1

64 / 64

parquet at cmd20.sc:1

24 / 24